In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:



                                            # Retail Sales Analysis




In [3]:
df1 = pd.read_csv("C:/Users/Nitesh/OneDrive/Desktop/TOPS TECHNOLOGY/Assessment/Python/Retail Data.csv")
print(df1)

     Order No  Order Date           Customer Name  \
0      4293-1  02-09-2014        Vivek Sundaresam   
1      5001-1  24-10-2015          Shahid Hopkins   
2      5004-1  13-03-2014           Dennis Pardue   
3      5009-1  18-02-2013              Sean Wendt   
4      5010-1  13-09-2014  Christina Vanderzanden   
...       ...         ...                     ...   
4995   6831-1  02-11-2016           Roland Murray   
4996   6847-1  16-12-2015          Patrick OBrill   
4997   6922-1  28-09-2016            Troy Staebel   
4998   6939-1  18-06-2016             Shaun Weien   
4999   6980-1  12-08-2015          Nora Pelletier   

                                  Address       City State   Customer Type  \
0          152 Bunnerong Road,Eastgardens     Sydney   NSW  Small Business   
1           438 Victoria Avenue,Chatswood     Sydney   NSW       Corporate   
2                412 Brunswick St,Fitzroy  Melbourne   VIC        Consumer   
3                145 Ramsay St,Haberfield     Sydne

In [4]:
#1. View the structure of the dataset (columns, types, missing values).

print(df1.columns)
print("\n",df1.dtypes)
print("\n",df1.isnull().sum())

Index(['Order No', 'Order Date', 'Customer Name', 'Address', 'City', 'State',
       'Customer Type', 'Account Manager', 'Order Priority', 'Product Name',
       'Product Category', 'Product Container', 'Ship Mode', 'Ship Date',
       'Cost Price', 'Retail Price', 'Profit Margin', 'Order Quantity',
       'Sub Total', 'Discount %', 'Discount $', 'Order Total', 'Shipping Cost',
       'Total'],
      dtype='object')

 Order No              object
Order Date            object
Customer Name         object
Address               object
City                  object
State                 object
Customer Type         object
Account Manager       object
Order Priority        object
Product Name          object
Product Category      object
Product Container     object
Ship Mode             object
Ship Date             object
Cost Price            object
Retail Price          object
Profit Margin         object
Order Quantity       float64
Sub Total             object
Discount %            objec

In [50]:
#2. What is the shape (rows, columns) of the dataset?

print(df1.shape)

(5000, 24)


In [5]:
top_5_customers = (
    df1.groupby(['Customer Name'])
      .agg(
          order_count=('Customer Name', 'count'),
          total_sales=('Total', 'sum')
      )
      .sort_values(by='order_count', ascending=False)
      .head(5)
)

print(top_5_customers)


                        order_count  \
Customer Name                         
Christina Vanderzanden           51   
Michael Oakman                   50   
Cindy Chapman                    44   
Dave Hallsten                    43   
Christopher Martinez             39   

                                                              total_sales  
Customer Name                                                              
Christina Vanderzanden  $125.97$129.40$129.57$129.57$129.57$129.57$53....  
Michael Oakman          $4,193.93$241.43$234.33$4,474.52$43.25$47.53$1...  
Cindy Chapman           $893.74$483.83$76.69$588.58$204.62$200.03$130....  
Dave Hallsten           $113.95$124.46$39.95$52.01$536.29$88.38$60.47$...  
Christopher Martinez    $128.52$96.00$60.88$6,529.86$105.00$9.82$9.82$...  


In [51]:

#3. Are there any duplicate records?\
print(df1.duplicated().sum())

"""No duplicate records"""

0


'No duplicate records'

In [52]:
#4.  Are there any missing or corrupted entries in Ship Date, Order Date, or numeric columns?

print(df1[["Ship Date", "Order Date"]].isnull().sum())

# THERE ARE NO MISSING VALUES OR CORRUPTED ENTRIES IN BOTH COLUMNS

Ship Date     0
Order Date    0
dtype: int64


In [53]:
#5.  Convert Order Date and Ship Date to datetime.

df1["Order Date"] = pd.to_datetime(df1["Order Date"], errors="coerce")
df1["Ship Date"] = pd.to_datetime(df1["Ship Date"], errors="coerce")

print(df1[["Order Date", "Ship Date"]].dtypes)

Order Date    datetime64[ns]
Ship Date     datetime64[ns]
dtype: object


In [54]:
#6.  Check for future or inconsistent shipping dates.

df1["Order Date"] = pd.to_datetime(df1["Order Date"], errors="coerce")
df1["Ship Date"] = pd.to_datetime(df1["Ship Date"], errors="coerce")

inconsistent_ship = df1[df1["Ship Date"] < df1["Order Date"]]

print("Number of inconsistent shipping dates (Ship Date < Order Date):")
print(len(inconsistent_ship))

today = pd.Timestamp.today()
future_ship = df1[df1["Ship Date"] > today]

print("\nNumber of future shipping dates:")
print(len(future_ship))

problematic_rows = pd.concat([inconsistent_ship, future_ship]).drop_duplicates()

print("\nSample problematic records:")
print(problematic_rows[["Order Date", "Ship Date"]].head())

Number of inconsistent shipping dates (Ship Date < Order Date):
24

Number of future shipping dates:
0

Sample problematic records:
     Order Date  Ship Date
204  2014-04-09 2014-03-09
764  2014-07-10 2014-02-10
765  2014-06-10 2014-02-10
863  2014-04-10 2014-02-10
1292 2014-12-02 2014-11-02


In [55]:
#7.  Convert price columns to numeric (remove $ and commas).

price_cols = ["Retail Price", "Profit Margin"]

# Convert price columns to numeric
for col in price_cols:
    df1[col] = (
        df1[col]
        .astype(str)  # ensure string type
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
    )
    df1[col] = pd.to_numeric(df1[col], errors="coerce")

print(df1[price_cols].dtypes)
# 8. What are the unique values in Customer Type and Order Priority?

customer_type_unique = df1["Customer Type"].unique()
order_priority_unique = df1["Order Priority"].unique()

print(customer_type_unique)
print(order_priority_unique)

Retail Price     float64
Profit Margin    float64
dtype: object
['Small Business' 'Corporate' 'Consumer' 'Home Office']
['Critical' 'Medium' 'Not Specified' 'Low' 'High']


In [56]:
# 8. What are the unique values in Customer Type and Order Priority?

customer_type_unique = df1["Customer Type"].unique()
order_priority_unique = df1["Order Priority"].unique()

print(customer_type_unique)
print(order_priority_unique)

['Small Business' 'Corporate' 'Consumer' 'Home Office']
['Critical' 'Medium' 'Not Specified' 'Low' 'High']


In [57]:
#9. What are the most common shipping modes?

print(df1["Ship Mode"].value_counts())

"Regular Air is the most common shipping mode"

Ship Mode
Regular Air       4236
Express Air        647
Delivery Truck     117
Name: count, dtype: int64


'Regular Air is the most common shipping mode'

In [58]:
# 10. Which cities have the highest number of orders?

df2 = df1.groupby("City")["Order Quantity"].count()
print(df2)

print("Melbourne has the highest number(1416) of orders")

City
Melbourne    1416
Sydney       3583
Name: Order Quantity, dtype: int64
Melbourne has the highest number(1416) of orders


In [59]:
# 11. What’s the range of order quantities and prices?

min_qty = df1["Order Quantity"].min()
max_qty = df1["Order Quantity"].max()

print(f"Order Quantity Range: {min_qty} to {max_qty}")

min_price = df1["Retail Price"].min()
max_price = df1["Retail Price"].max()

print(f"Price (Sales) Range: {min_price} to {max_price}")

Order Quantity Range: 1.0 to 50.0
Price (Sales) Range: 1.14 to 599.99


In [60]:
# 12. Create a new column for shipping duration.

df1["Shipping Duration"] = df1["Ship Date"] - df1["Order Date"]
print(df1)

     Order No Order Date           Customer Name  \
0      4293-1 2014-02-09        Vivek Sundaresam   
1      5001-1        NaT          Shahid Hopkins   
2      5004-1        NaT           Dennis Pardue   
3      5009-1        NaT              Sean Wendt   
4      5010-1        NaT  Christina Vanderzanden   
...       ...        ...                     ...   
4995   6831-1 2016-02-11           Roland Murray   
4996   6847-1        NaT          Patrick OBrill   
4997   6922-1        NaT            Troy Staebel   
4998   6939-1        NaT             Shaun Weien   
4999   6980-1 2015-12-08          Nora Pelletier   

                                  Address       City State   Customer Type  \
0          152 Bunnerong Road,Eastgardens     Sydney   NSW  Small Business   
1           438 Victoria Avenue,Chatswood     Sydney   NSW       Corporate   
2                412 Brunswick St,Fitzroy  Melbourne   VIC        Consumer   
3                145 Ramsay St,Haberfield     Sydney   NSW  Sma

In [61]:
# 13. Are there any orders with zero or negative total or quantity?

price_cols = ["Order Total"]

for col in price_cols:
    df1[col] = (
        df1[col]
        .astype(str)  # ensure string type
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
    )
    df1[col] = pd.to_numeric(df1[col], errors="coerce")

zero_negative_qty = df1[df1["Order Quantity"] <= 0]

print("Orders with zero or negative quantity:")
print(len(zero_negative_qty))

zero_negative_sales = df1[df1["Order Total"] <= 0]

print("\nOrders with zero or negative total (Sales):")
print(len(zero_negative_sales))

Orders with zero or negative quantity:
0

Orders with zero or negative total (Sales):
0


In [62]:
# 14. Are all discount percentages matching discount dollar amounts?

# Clean Discount %
df1["Discount %"] = pd.to_numeric(
    df1["Discount %"].astype(str).str.replace("%", "", regex=False),
    errors="coerce"
)

# Clean Discount $
df1["Discount $"] = pd.to_numeric(
    df1["Discount $"].astype(str).str.replace("[$,]", "", regex=True),
    errors="coerce"
)

# Clean Sub Total
df1["Sub Total"] = pd.to_numeric(
    df1["Sub Total"].astype(str).str.replace("[$,]", "", regex=True),
    errors="coerce"
)

df1["Expected Discount $"] = df1["Sub Total"] * df1["Discount %"] / 100
df1["Discount Match"] = (
        abs(df1["Discount $"] - df1["Expected Discount $"]) <= 1)

df1["Discount Match"].value_counts()

Discount Match
False    3975
True     1025
Name: count, dtype: int64

In [63]:
# 15.  Check for mismatches in total calculation.

cols = ["Sub Total", "Discount $", "Shipping Cost", "Total"]

for col in cols:
    df1[col] = pd.to_numeric(
        df1[col].astype(str).str.replace("[$,]", "", regex=True),
        errors="coerce")

df1["Expected Total"] = (
        df1["Sub Total"] - df1["Discount $"] + df1["Shipping Cost"]
)

df1["Total Match"] = (
        abs(df1["Total"] - df1["Expected Total"]) <= 1
)

df1["Total Match"].value_counts()

Total Match
False    3235
True     1765
Name: count, dtype: int64

In [64]:
# 16.  Identify top 5 products by order quantity.

print("Top 5 Products by Total Order Quantity:\n")
df1.groupby('Product Name')['Order Quantity'].sum().sort_values(ascending=False).head()

Top 5 Products by Total Order Quantity:



Product Name
Artisan 474 Labels                         3395.0
Smiths General Use 3-Ring Binders          3220.0
Artisan 479 Labels                         3039.0
Artisan 481 Labels                         2996.0
Apex Preferred Stainless Steel Scissors    2896.0
Name: Order Quantity, dtype: float64

In [65]:
# 17. Which Account Manager handled the most revenue?

revenue_by_manager = df1.groupby('Account Manager')['Order Total'].sum().reset_index()

top_managers = revenue_by_manager.sort_values(by='Order Total', ascending=False)

top_5 = top_managers.head(5)

print("Top 5 Account Managers by Total Revenue:\n")
print(top_5.to_string(index=False))

# Identify the one with the most revenue
top_manager = top_5.iloc[0]
print(f"\nAccount Manager who handled the most revenue: {top_manager['Account Manager']}")
print(f"Total Revenue: ${top_manager['Order Total']:,.2f}")

Top 5 Account Managers by Total Revenue:

Account Manager  Order Total
   Connor Betts    746122.56
   Tina Carlton    447203.42
    Yvette Biti    388451.58
Samantha Chairs    282291.69
    Mihael Khan    280690.79

Account Manager who handled the most revenue: Connor Betts
Total Revenue: $746,122.56


In [66]:
#18.  What is the average shipping cost by mode?

price_cols = ["Shipping Cost"]
for col in price_cols:
    df1[col] = (
        df1[col]
        .astype(str)  # ensure string type
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
    )
    df1[col] = pd.to_numeric(df1[col], errors="coerce")

df1.groupby('Ship Mode')['Shipping Cost'].mean()

Ship Mode
Delivery Truck    5.270085
Express Air       5.247311
Regular Air       5.503553
Name: Shipping Cost, dtype: float64

In [67]:
# 19. Find the most profitable product.

most_profitable_product = df1.groupby("Product Name")["Profit Margin"].mean()
# print(most_profitable_product)

most_profitable_product_name = most_profitable_product.idxmax()

print("Most profitable product:", most_profitable_product_name)

Most profitable product: 3wax Printer


In [24]:
# 20. Find the most profitable product.

most_profitable_product = df1.groupby("Product Name")["Profit Margin"].mean()
# print(most_profitable_product)

most_profitable_product_name = most_profitable_product.idxmax()

print("Most profitable product:", most_profitable_product_name)

Most profitable product: 3wax Printer


In [ ]:


                                         # Insightful Analysis Questions




In [25]:
import pandas as pd

df1 = pd.read_csv("C:/Users/Nitesh/OneDrive/Desktop/TOPS TECHNOLOGY/Assessment/Python/Retail Data.csv")
print(df1)

     Order No  Order Date           Customer Name  \
0      4293-1  02-09-2014        Vivek Sundaresam   
1      5001-1  24-10-2015          Shahid Hopkins   
2      5004-1  13-03-2014           Dennis Pardue   
3      5009-1  18-02-2013              Sean Wendt   
4      5010-1  13-09-2014  Christina Vanderzanden   
...       ...         ...                     ...   
4995   6831-1  02-11-2016           Roland Murray   
4996   6847-1  16-12-2015          Patrick OBrill   
4997   6922-1  28-09-2016            Troy Staebel   
4998   6939-1  18-06-2016             Shaun Weien   
4999   6980-1  12-08-2015          Nora Pelletier   

                                  Address       City State   Customer Type  \
0          152 Bunnerong Road,Eastgardens     Sydney   NSW  Small Business   
1           438 Victoria Avenue,Chatswood     Sydney   NSW       Corporate   
2                412 Brunswick St,Fitzroy  Melbourne   VIC        Consumer   
3                145 Ramsay St,Haberfield     Sydne

In [26]:
# 1. What is the total revenue generated across all orders?

price_cols = ["Total"]
for col in price_cols:
    df1[col] = (
        df1[col]
        .astype(str)  # ensure string type
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
    )
    df1[col] = pd.to_numeric(df1[col], errors="coerce")

total_revenue = df1["Total"].sum()
print(total_revenue)

3731536.4000000004


In [27]:
# 2. Which customer type generates more revenue?

price_cols = ["Order Total"]
for col in price_cols:
    df1[col] = (
        df1[col]
        .astype(str)  # ensure string type
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
    )
    df1[col] = pd.to_numeric(df1[col], errors="coerce")

customer_revenue = df1.groupby('Customer Type')['Order Total'].sum().sort_values(ascending=False)
print(customer_revenue)

""" Corporate generates more revenue than any other customer types"""

Customer Type
Corporate         1352669.38
Home Office        871795.73
Small Business     849685.95
Consumer           734848.79
Name: Order Total, dtype: float64


' Corporate generates more revenue than any other customer types'

In [29]:
# 3.  How does order priority affect revenue?

revenue_by_priority = (
    df1.groupby("Order Priority")["Total"].sum().sort_values(ascending=False))

print(revenue_by_priority)

Order Priority
Not Specified    817844.66
High             794199.00
Medium           784744.63
Critical         694547.97
Low              640200.14
Name: Total, dtype: float64


In [31]:
# 4. What is the average profit margin by product category?

df1["Profit Margin"] = pd.to_numeric(
    df1["Profit Margin"].str.replace("[$,]", "", regex=True),
    errors="coerce")

avg_profit_margin = df1.groupby("Product Category")["Profit Margin"].mean().reset_index()
print(avg_profit_margin)

  Product Category  Profit Margin
0        Furniture      12.701953
1  Office Supplies      17.682609
2       Technology      20.423208


In [32]:
# 5. What is the most profitable product overall?

most_profitable_product = df1.groupby('Product Name')['Profit Margin'].sum().idxmax()
print("Most profitable product:", most_profitable_product)

Most profitable product: Cando PC940 Copier


In [33]:
# 6. How many days does it usually take to ship an order?

df1["Ship Date"] = pd.to_datetime(df1["Ship Date"], dayfirst=True)
df1["Order Date"] = pd.to_datetime(df1["Order Date"], dayfirst=True)

df1["Shipping Duration"] = df1["Ship Date"] - df1["Order Date"]

avg_shipping_time = df1["Shipping Duration"].mean()
print(avg_shipping_time)

print("It usually takes 1 day to ship an order")

1 days 20:03:15.840000
It usually takes 1 day to ship an order


In [34]:
# 7. Do longer shipping times impact profit margins?

correlation = df1["Shipping Duration"].corr(df1["Profit Margin"])
print(correlation)

print("YEs, longer shipping times hurt profit margin")

-0.01978703518675127
YEs, longer shipping times hurt profit margin


In [35]:
# 8. Which city brings in the highest revenue?

print(df1.groupby("City")["Total"].sum())   # to compare
print(df1.groupby("City")["Total"].sum().idxmax())

print("\nSydney brings in highest revenue")

City
Melbourne    1052756.07
Sydney       2678780.33
Name: Total, dtype: float64
Sydney

Sydney brings in highest revenue


In [36]:
# 9. Which account manager generated the most revenue?

print(df1.groupby("Account Manager")["Total"].sum())  # to see if it actually generates highest revenue
print()
print(df1.groupby("Account Manager")["Total"].sum().idxmax())

print("Conner Betts generated highest revenue")

Account Manager
Aanya Zhang           241344.03
Charlie Bui           138153.12
Charlie Buo                5.07
Connor Betts          671173.17
Eanya Zhang              240.69
Leighton Forrest      216736.14
Mihael Khan           281948.64
Natasha Song          242718.26
Nicholas Fernandes    305927.88
Oanya Zhang               73.57
Phoebe Gour           249340.81
Preston Senome        199052.07
Radhya Staples         61519.53
Samantha Chairs       273750.35
Simantha Chairs           55.12
Stevie Bacata          15879.22
Tina Carlton          453891.62
Wicholas Fernandes        39.99
Yvette Biti           379687.12
Name: Total, dtype: float64

Connor Betts
Conner Betts generated highest revenue


In [37]:
# 10.  Which shipping mode is most cost-effective (lowest avg. shipping)?

print(df1.groupby("Ship Mode")["Total"].mean())
print()
print(df1.groupby("Ship Mode")["Total"].mean().idxmin())

Ship Mode
Delivery Truck    607.649573
Express Air       693.945456
Regular Air       758.134724
Name: Total, dtype: float64

Delivery Truck


In [39]:
# 11.  Do higher discounts reduce profits?

df1["Discount $"] = pd.to_numeric(
    df1["Discount $"].str.replace("[$,]", "", regex=True),
    errors="coerce"
)

correlation = df1["Discount $"].corr(df1["Profit Margin"])
print(correlation)

print(f"Yes, longer shipping times hurt profit margin but not that much")

0.39349726173151367
Yes, longer shipping times hurt profit margin but not that much


In [40]:
# 12. Which state has the highest number of orders?

print(df1.groupby("State")["Order Quantity"].count())
print()
print(df1.groupby("State")["Order Quantity"].count().idxmax())

print("NSW has the highest number(3583) of orders")

State
NSW    3583
VIC    1416
Name: Order Quantity, dtype: int64

NSW
NSW has the highest number(3583) of orders


In [41]:
# 13. What is the average discount % across all orders?

print(df1["Discount $"].mean())
print("Average discount across all orders is 49.43")

49.4329
Average discount across all orders is 49.43


In [42]:
# 14. What is the average total spend per order?

avg_spending = df1["Total"].mean()
print(avg_spending)
print(f"Average total spending per order is {avg_spending}")

746.3072800000001
Average total spending per order is 746.3072800000001


In [43]:
# 15. Are certain containers (e.g., Small Box, Wrap Bag) more profitable?

container_profit = (
    df1.groupby("Product Container")["Profit Margin"]
    .mean()
    .sort_values(ascending=False)
)

print(container_profit)

print("\nSmall Box Containers have more profit margin. So yes, certain containers are more profitable")


Product Container
Small Box     19.668452
Small Pack    16.836970
Jumbo Drum    16.483419
Wrap Bag      16.407018
Medium Box    15.637523
Large Box     11.637730
Name: Profit Margin, dtype: float64

Small Box Containers have more profit margin. So yes, certain containers are more profitable
